# Cleaning Crystal Structure

In [3]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

pdbCode = "4rc7"
ligandCode = "pl3"
mol_charge = 0

In [10]:
view = nglview.show_structure_file(f"input/{pdbCode}.pdb")
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', radius='0.1', selection='water')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ligand')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ion')
view._remote_call('setSize', target='Widget', args=['','600px'])

view

NGLWidget()

In [11]:
from biobb_structure_utils.utils.remove_pdb_water import remove_pdb_water

nowat_pdb = pdbCode+'.nowat.pdb'

remove_pdb_water(input_pdb_path=f"input/{pdbCode}.pdb",
    output_pdb_path=f"input/{nowat_pdb}")

2022-06-29 22:27:58,026 [MainThread  ] [INFO ]  check_structure -i input/4rc7.pdb -o input/4rc7.nowat.pdb --force_save water --remove yes

2022-06-29 22:27:58,029 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:27:58,030 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure input/4rc7.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  318
 Num. residues with ins. codes:  0
 Num. HETATM residues:  98
 Num. ligands or modified residues:  3
 Num. water mol.:  95
 Num. atoms:  1881
Metal/Ion residues found
FE2 A1002
FE2 A1003
Small mol ligands found
PL3 A1001

Running water. Options: --remove yes
95 Water molecules detected
95 Water molecules removed
Final Num. models: 1
Final Num. chain

0

In [12]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

nofe2_pdb = pdbCode+'.noFe2.pdb'

prop = {
    'ligand' : 'FE2'
}

remove_ligand(input_structure_path=f"input/{nowat_pdb}",
    output_structure_path=f"input/{nofe2_pdb}",
    properties=prop)

2022-06-29 22:27:58,087 [MainThread  ] [INFO ]  PDB format detected, removing all atoms from residues named FE2


0

In [13]:
view = nglview.show_structure_file(f"input/{nofe2_pdb}")
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', radius='0.1', selection='water')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ligand')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ion')
view._remote_call('setSize', target='Widget', args=['','600px'])

view

NGLWidget()

# Ligand Topology

In [14]:
# Import module
from biobb_amber.pdb4amber.pdb4amber_run import pdb4amber_run

# Crea# Cleaning Crystal Structurete prop dict and inputs/outputs
output_pdb4amber_path = 'structure.pdb4amber.pdb'

# Create and launch bb
pdb4amber_run(input_pdb_path=f"input/{nofe2_pdb}",
            output_pdb_path=f"input/{output_pdb4amber_path}")

2022-06-29 22:27:58,319 [MainThread  ] [INFO ]  Creating 12775723-c7a6-4d42-b1f1-cfc81268ba40 temporary folder
2022-06-29 22:27:58,320 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:27:59,384 [MainThread  ] [INFO ]  pdb4amber  -i input/4rc7.noFe2.pdb -o input/structure.pdb4amber.pdb

2022-06-29 22:27:59,386 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:27:59,388 [MainThread  ] [INFO ]  
Summary of pdb4amber for: input/4rc7.noFe2.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames
PL3

---------- Mising heavy atom(s)

None

2022-06-29 22:27:59,390 [MainThread  ] [INFO ]  Removed: 12775723-c7a6-4d42-b1f1-cfc81268ba40


0

In [15]:
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

ligandFile = ligandCode+'.pdb'

prop = {
     'heteroatoms' : [{"name": "PL3"}]
}

extract_heteroatoms(input_structure_path=f"input/{output_pdb4amber_path}",
     output_heteroatom_path=f"input/{ligandFile}",
     properties=prop)

2022-06-29 22:27:59,554 [MainThread  ] [INFO ]  File input/pl3.pdb created


0

In [16]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

output_reduce_h = ligandCode+'.reduce.H.pdb' 

prop = {
    'build': True,
    'flip': True,
    'rotexist': True,
    'nuclear': True
}

reduce_add_hydrogens(input_path=f"input/{ligandFile}",
                   output_path=f"input/{output_reduce_h}",
                   properties=prop)

2022-06-29 22:27:59,609 [MainThread  ] [INFO ]  Not using any container
2022-06-29 22:28:00,231 [MainThread  ] [INFO ]  reduce -FLIP -NUClear -OH -ROTNH3 -ROTEXist -ALLALT -BUILD input/pl3.pdb > input/pl3.reduce.H.pdb

2022-06-29 22:28:00,233 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:28:00,234 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "input/pl3.pdb"
Database of HETATM connections: "/Users/ethan/anaconda3/envs/biobb_AMBER_MD//dat/reduce_wwPDB_het_dict.txt                                                                                                                                                                                                                     "
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hy

0

In [17]:
from biobb_chemistry.babelm.babel_minimize import babel_minimize

output_babel_min = ligandCode+'.H.min.mol2'

prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}

babel_minimize(input_path=f"input/{output_reduce_h}",
              output_path=f"input/{output_babel_min}",
              properties=prop)

2022-06-29 22:28:00,324 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2022-06-29 22:28:00,326 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2022-06-29 22:28:00,328 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2022-06-29 22:28:00,330 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2022-06-29 22:28:00,333 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2022-06-29 22:28:00,336 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2022-06-29 22:28:00,337 [MainThread  ] [INFO ]  Not using any container
2022-06-29 22:28:02,650 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb input/pl3.reduce.H.pdb -omol2 > input/pl3.H.min.mol2

2022-06-29 22:28:02,656 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:28:02,659 [MainThread  ] [INFO ]  
A T O M   T Y P E S

IDX	TYPE	RING
1	c3	NO
2	c3	NO
3	c3	NO
4	c3	NO
5	c

0

In [18]:
view1 = nglview.show_structure_file(f"input/{ligandFile}")
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'

view1
view2 = nglview.show_structure_file(f"input/{output_reduce_h}")
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'

view2
view3 = nglview.show_structure_file(f"input/{output_babel_min}")
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'

view3
ipywidgets.HBox([view1, view2, view3])

In [20]:
from biobb_chemistry.acpype.acpype_params_ac import acpype_params_ac

output_acpype_inpcrd = ligandCode+'params.inpcrd'
output_acpype_frcmod = ligandCode+'params.frcmod'
output_acpype_lib = ligandCode+'params.lib'
output_acpype_prmtop = ligandCode+'params.prmtop'
output_acpype = ligandCode+'params'

prop = {
    'basename' : output_acpype,
    'charge' : None
}

acpype_params_ac(input_path=f"input/{output_babel_min}", 
                output_path_inpcrd=f"input/{output_acpype_inpcrd}",
                output_path_frcmod=f"input/{output_acpype_frcmod}",
                output_path_lib=f"input/{output_acpype_lib}",
                output_path_prmtop=f"input/{output_acpype_prmtop}",
                properties=prop)

2022-06-29 22:29:19,823 [MainThread  ] [INFO ]  Charge will be guessed by acpype.
2022-06-29 22:29:19,824 [MainThread  ] [INFO ]  Running acpype, this execution can take a while
2022-06-29 22:29:19,825 [MainThread  ] [INFO ]  Not using any container
2022-06-29 22:30:05,405 [MainThread  ] [INFO ]  acpype -i /Users/ethan/EthanLatchProj/input/pl3.H.min.mol2 -b pl3params.9AkUW3

2022-06-29 22:30:05,407 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:30:05,408 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2022 AWSdS |
==> ... charge set to -1
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'PL3'....

/Users/ethan/anaconda3/envs/biobb_AMBER_MD/bin/teLeap: Warning!
The unperturbed charge of the unit (-0.998999) is n

0

# Complex Topology

In [21]:
from biobb_amber.leap.leap_gen_top import leap_gen_top

output_pdb_path = 'structure.leap.pdb'
output_top_path = 'structure.leap.top'
output_crd_path = 'structure.leap.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"]
}

leap_gen_top(input_pdb_path=f"input/{output_pdb4amber_path}",
           input_lib_path=f"input/{output_acpype_lib}",
           input_frcmod_path=f"input/{output_acpype_frcmod}",
           output_pdb_path=f"input/{output_pdb_path}",
           output_top_path=f"input/{output_top_path}",
           output_crd_path=f"input/{output_crd_path}",
           properties=prop)

2022-06-29 22:30:10,137 [MainThread  ] [INFO ]  Creating 001df548-a14d-4830-93ce-d3c04da5336e temporary folder
2022-06-29 22:30:10,139 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:30:10,494 [MainThread  ] [INFO ]  tleap  -f 001df548-a14d-4830-93ce-d3c04da5336e/leap.in

2022-06-29 22:30:10,496 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:30:10,496 [MainThread  ] [INFO ]  -I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/prep to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/lib to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/parm to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd to search path.
-f: Source 001df548-a14d-4830-93ce-d3c04da5336e/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./001df548-a14d-4830-93ce-d3c04da5336e/leap.in
----- Source: /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd/le

0

In [25]:
view = nglview.show_structure_file(f"input/{output_pdb_path}")
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', opacity='1')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='PL3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

# Energy Minimization

In [26]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

output_h_min_traj_path = 'sander.h_min.x'
output_h_min_rst_path = 'sander.h_min.rst'
output_h_min_log_path = 'sander.h_min.log'

prop = {
    'simulation_type' : "min_vacuo",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'ntr' : 1,
        'restraintmask' : '\":*&!@H=\"',
        'restraint_wt' : 50.0
    }
}

sander_mdrun(input_top_path=f"input/{output_top_path}",
            input_crd_path=f"input/{output_crd_path}",
            input_ref_path=f"input/{output_crd_path}",
            output_traj_path=f"output/{output_h_min_traj_path}",
            output_rst_path=f"output/{output_h_min_rst_path}",
            output_log_path=f"output/{output_h_min_log_path}",
            properties=prop)

2022-06-29 22:31:11,446 [MainThread  ] [INFO ]  Creating 8ad6f294-80fd-4659-8a99-3ae9a8b2ddbb temporary folder
2022-06-29 22:31:11,447 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:31:33,613 [MainThread  ] [INFO ]  sander -O -i 8ad6f294-80fd-4659-8a99-3ae9a8b2ddbb/sander.mdin -p input/structure.leap.top -c input/structure.leap.crd -r output/sander.h_min.rst -o output/sander.h_min.log -x output/sander.h_min.x -ref input/structure.leap.crd

2022-06-29 22:31:33,615 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:31:33,617 [MainThread  ] [INFO ]  Removed: mdinfo, 8ad6f294-80fd-4659-8a99-3ae9a8b2ddbb


0

In [27]:
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_h_min_dat_path = 'sander.h_min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=f"output/{output_h_min_log_path}",
            output_dat_path=f"output/{output_h_min_dat_path}",
            properties=prop)  

2022-06-29 22:31:47,111 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:31:47,196 [MainThread  ] [INFO ]  process_minout.perl  output/sander.h_min.log

2022-06-29 22:31:47,198 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:31:47,199 [MainThread  ] [INFO ]  Processing sander output file (output/sander.h_min.log)...
Processing step 50 of a possible 500...
Processing step 100 of a possible 500...
Processing step 150 of a possible 500...
Processing step 200 of a possible 500...
Processing step 250 of a possible 500...
Processing step 300 of a possible 500...
Processing step 350 of a possible 500...
Processing step 400 of a possible 500...
Processing step 450 of a possible 500...
Processing step 500 of a possible 500...
Processing step 500 of a possible 500...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summar

0

In [28]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(f"output/{output_h_min_dat_path}",'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

# Creating Solvent Box

In [29]:
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

output_ambpdb_path = 'structure.ambpdb.pdb'

amber_to_pdb(input_top_path=f"input/{output_top_path}",
            input_crd_path=f"output/{output_h_min_rst_path}",
            output_pdb_path=f"input/{output_ambpdb_path}")

2022-06-29 22:33:01,155 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:33:01,320 [MainThread  ] [INFO ]  ambpdb  -p input/structure.leap.top -c output/sander.h_min.rst >  input/structure.ambpdb.pdb

2022-06-29 22:33:01,323 [MainThread  ] [INFO ]  Exit code 0



0

In [30]:
from biobb_amber.leap.leap_solvate import leap_solvate

output_solv_pdb_path = 'structure.solv.pdb'
output_solv_top_path = 'structure.solv.parmtop'
output_solv_crd_path = 'structure.solv.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"],
    "water_type": "TIP3PBOX",
    "distance_to_molecule": "9.0",   
    "box_type": "truncated_octahedron"
}

leap_solvate(input_pdb_path=f"input/{output_ambpdb_path}",
             input_lib_path=f"input/{output_acpype_lib}",
             input_frcmod_path=f"input/{output_acpype_frcmod}",
             output_pdb_path=f"input/{output_solv_pdb_path}",
             output_top_path=f"input/{output_solv_top_path}",
             output_crd_path=f"input/{output_solv_crd_path}",
             properties=prop)

2022-06-29 22:33:02,422 [MainThread  ] [INFO ]  Creating 9ac7028a-8df2-4061-abf2-db1be9378638 temporary folder
2022-06-29 22:33:02,425 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:33:03,799 [MainThread  ] [INFO ]  tleap  -f 9ac7028a-8df2-4061-abf2-db1be9378638/leap.in

2022-06-29 22:33:03,802 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:33:03,806 [MainThread  ] [INFO ]  -I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/prep to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/lib to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/parm to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd to search path.
-f: Source 9ac7028a-8df2-4061-abf2-db1be9378638/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./9ac7028a-8df2-4061-abf2-db1be9378638/leap.in
----- Source: /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd/le

0

In [31]:
from biobb_amber.leap.leap_add_ions import leap_add_ions

output_ions_pdb_path = 'structure.ions.pdb'
output_ions_top_path = 'structure.ions.parmtop'
output_ions_crd_path = 'structure.ions.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"],
    "neutralise" : True,
    "positive_ions_type": "Na+",
    "negative_ions_type": "Cl-",
    "ionic_concentration" : 150, # 150mM
    "box_type": "truncated_octahedron"
}

leap_add_ions(input_pdb_path=f"input/{output_solv_pdb_path}",
            input_lib_path=f"input/{output_acpype_lib}",
            input_frcmod_path=f"input/{output_acpype_frcmod}",
           output_pdb_path=f"input/{output_ions_pdb_path}",
           output_top_path=f"input/{output_ions_top_path}",
           output_crd_path=f"input/{output_ions_crd_path}",
           properties=prop)

2022-06-29 22:33:08,165 [MainThread  ] [INFO ]  Creating 3a089f7d-de82-4696-a33e-ff0ce1333c58 temporary folder
2022-06-29 22:33:08,255 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:33:10,109 [MainThread  ] [INFO ]  tleap  -f 3a089f7d-de82-4696-a33e-ff0ce1333c58/leap.in

2022-06-29 22:33:10,111 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:33:10,112 [MainThread  ] [INFO ]  -I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/prep to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/lib to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/parm to search path.
-I: Adding /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd to search path.
-f: Source 3a089f7d-de82-4696-a33e-ff0ce1333c58/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./3a089f7d-de82-4696-a33e-ff0ce1333c58/leap.in
----- Source: /Users/ethan/anaconda3/envs/biobb_AMBER_MD/dat/leap/cmd/le

0

In [32]:
view = nglview.show_structure_file(f"input/{output_ions_pdb_path}")
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein')
view.add_representation(repr_type='ball+stick', selection='solvent')
view.add_representation(repr_type='spacefill', selection='Cl- Na+')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

# Minimize Solvated Complex

In [33]:
output_min_traj_path = 'sander.min.x'
output_min_rst_path = 'sander.min.rst'
output_min_log_path = 'sander.min.log'

prop = {
    "simulation_type" : "minimization",
    "mdin" : { 
        'maxcyc' : 300, # Reducing the number of minimization steps for the sake of time
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 15.0                       # With a force constant of 50 Kcal/mol*A2
    }
}

sander_mdrun(input_top_path=f"input/{output_ions_top_path}",
            input_crd_path=f"input/{output_ions_crd_path}",
            input_ref_path=f"input/{output_ions_crd_path}",
            output_traj_path=f"output/{output_min_traj_path}",
            output_rst_path=f"output/{output_min_rst_path}",
            output_log_path=f"output/{output_min_log_path}",
            properties=prop)

2022-06-29 22:33:15,633 [MainThread  ] [INFO ]  Creating 9e144d97-d767-4adb-826b-a355da17a80c temporary folder
2022-06-29 22:33:15,634 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:34:38,282 [MainThread  ] [INFO ]  sander -O -i 9e144d97-d767-4adb-826b-a355da17a80c/sander.mdin -p input/structure.ions.parmtop -c input/structure.ions.crd -r output/sander.min.rst -o output/sander.min.log -x output/sander.min.x -ref input/structure.ions.crd

2022-06-29 22:34:38,286 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:34:38,289 [MainThread  ] [INFO ]  Removed: mdinfo, 9e144d97-d767-4adb-826b-a355da17a80c


0

In [34]:
output_dat_path = 'sander.min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

process_minout(input_log_path=f"output/{output_min_log_path}",
            output_dat_path=f"output/{output_dat_path}",
            properties=prop)

2022-06-29 22:34:38,354 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:34:38,418 [MainThread  ] [INFO ]  process_minout.perl  output/sander.min.log

2022-06-29 22:34:38,422 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:34:38,424 [MainThread  ] [INFO ]  Processing sander output file (output/sander.min.log)...
Processing step 150 of a possible 300...
Processing step 300 of a possible 300...
Processing step 300 of a possible 300...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputing summary.ANGLE
Outputing summary.DIHEDRAL
Outputing summary.VDWAALS
Outputing summary.EEL
Outputing summary.HBOND
Outputing summary.VDW14
Outputing summary.EEL14
Outputing summary.RESTRAINT

2022-06-29 22:34:38,431 [MainThread  ] [INFO ]  Removed: [PosixPath('summary.ENERGY'), PosixPath('summary.VDW14'), PosixPath

0

In [5]:
with open(f"output/sander.min.energy.dat",'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

# Heating System

In [39]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

output_heat_traj_path = 'sander.heat.netcdf'
output_heat_rst_path = 'sander.heat.rst'
output_heat_log_path = 'sander.heat.log'

prop = {
    "simulation_type" : "heat",
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (5ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 10.0                       # With a force constant of 10 Kcal/mol*A2
    }
}

sander_mdrun(input_top_path=f"input/{output_ions_top_path}",
            input_crd_path=f"output/{output_min_rst_path}",
            input_ref_path=f"output/{output_min_rst_path}",
            output_traj_path=f"output/{output_heat_traj_path}",
            output_rst_path=f"output/{output_heat_rst_path}",
            output_log_path=f"output/{output_heat_log_path}",
            properties=prop)

2022-06-29 22:41:26,855 [MainThread  ] [INFO ]  Creating ff8f03c0-5bb4-4d61-ab94-6d267ed6ced6 temporary folder
2022-06-29 22:41:26,857 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:45:31,837 [MainThread  ] [INFO ]  sander -O -i ff8f03c0-5bb4-4d61-ab94-6d267ed6ced6/sander.mdin -p input/structure.ions.parmtop -c output/sander.min.rst -r output/sander.heat.rst -o output/sander.heat.log -x output/sander.heat.netcdf -ref output/sander.min.rst

2022-06-29 22:45:31,840 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:45:31,843 [MainThread  ] [INFO ]  Removed: mdinfo, ff8f03c0-5bb4-4d61-ab94-6d267ed6ced6


0

In [42]:
from biobb_amber.process.process_mdout import process_mdout

output_dat_heat_path = 'sander.md.temp.dat'

prop = {
"terms" : ['TEMP']
}

process_mdout(input_log_path=f"output/{output_heat_log_path}",
            output_dat_path=f"output/{output_dat_heat_path}",
            properties=prop)

2022-06-29 22:47:58,411 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-06-29 22:47:58,470 [MainThread  ] [INFO ]  process_mdout.perl  output/sander.heat.log

2022-06-29 22:47:58,473 [MainThread  ] [INFO ]  Exit code 0

2022-06-29 22:47:58,475 [MainThread  ] [INFO ]  Processing sander output file (output/sander.heat.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg

0

In [4]:
output_dat_heat_path = 'sander.md.temp.dat'

with open(f"output/{output_dat_heat_path}",'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Heating process",
                        xaxis=dict(title = "Heating Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

# Equilibrate Temp

In [6]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

prop = {
    "simulation_type" : 'nvt',
    "mdin" : { 
        'nstlim' : 100, # Reducing the number of steps for the sake of time (0.2ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0                              # With a force constant of 5 Kcal/mol*A2
    }
}

sander_mdrun(input_top_path="input/structure.ions.parmtop",
            input_crd_path="output/sander.heat.rst",
            input_ref_path="output/sander.heat.rst",
            output_traj_path="output/sander.nvt.netcdf",
            output_rst_path="output/sander.nvt.rst",
            output_log_path="output/sander.nvt.log",
            properties=prop)

2022-07-08 12:11:00,261 [MainThread  ] [INFO ]  Creating b991f5a0-f11e-4aca-a902-96aed3110716 temporary folder
2022-07-08 12:11:00,265 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-07-08 12:11:49,097 [MainThread  ] [INFO ]  sander -O -i b991f5a0-f11e-4aca-a902-96aed3110716/sander.mdin -p input/structure.ions.parmtop -c output/sander.heat.rst -r output/sander.nvt.rst -o output/sander.nvt.log -x output/sander.nvt.netcdf -ref output/sander.heat.rst

2022-07-08 12:11:49,099 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:11:49,102 [MainThread  ] [INFO ]  Removed: mdinfo, b991f5a0-f11e-4aca-a902-96aed3110716


0

In [7]:
from biobb_amber.process.process_mdout import process_mdout

prop = {
    "terms" : ['TEMP']
}

process_mdout(input_log_path="output/sander.nvt.log",
            output_dat_path="output/sander.nvt.temp.dat",
            properties=prop)

2022-07-08 12:24:04,088 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-07-08 12:24:04,275 [MainThread  ] [INFO ]  process_mdout.perl  output/sander.nvt.log

2022-07-08 12:24:04,278 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:24:04,279 [MainThread  ] [INFO ]  Processing sander output file (output/sander.nvt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.E

0

In [8]:
with open("output/sander.nvt.temp.dat",'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="NVT equilibration",
                        xaxis=dict(title = "Equilibration Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

# Equilibrate Pressure

In [9]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

prop = {
    "simulation_type" : 'npt',
    "mdin" : { 
        'nstlim' : 100, # Reducing the number of steps for the sake of time (0.2ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0                              # With a force constant of 5 Kcal/mol*A2
    }
}

sander_mdrun(input_top_path="input/structure.ions.parmtop",
            input_crd_path="output/sander.nvt.rst",
            input_ref_path="output/sander.nvt.rst",
            output_traj_path="output/sander.npt.netcdf",
            output_rst_path="output/sander.npt.rst",
            output_log_path="output/sander.npt.log",
            properties=prop)

2022-07-08 12:26:08,929 [MainThread  ] [INFO ]  Creating 0a13a4ab-d171-453a-8d68-c4d4ba389ff1 temporary folder
2022-07-08 12:26:08,930 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-07-08 12:26:58,977 [MainThread  ] [INFO ]  sander -O -i 0a13a4ab-d171-453a-8d68-c4d4ba389ff1/sander.mdin -p input/structure.ions.parmtop -c output/sander.nvt.rst -r output/sander.npt.rst -o output/sander.npt.log -x output/sander.npt.netcdf -ref output/sander.nvt.rst

2022-07-08 12:26:58,979 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:26:58,982 [MainThread  ] [INFO ]  Removed: mdinfo, 0a13a4ab-d171-453a-8d68-c4d4ba389ff1


0

In [10]:
from biobb_amber.process.process_mdout import process_mdout

prop = {
    "terms" : ['PRES','DENSITY']
}

process_mdout(input_log_path="output/sander.npt.log",
            output_dat_path="output/sander.npt.dat",
            properties=prop)

2022-07-08 12:27:56,049 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-07-08 12:27:56,080 [MainThread  ] [INFO ]  process_mdout.perl  output/sander.npt.log

2022-07-08 12:27:56,082 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:27:56,084 [MainThread  ] [INFO ]  Processing sander output file (output/sander.npt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.E

0

In [11]:
with open("output/sander.npt.dat",'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

# Running MD

In [12]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

prop = {
    "simulation_type" : 'free',
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (1ps)
        'ntwx' : 100  # Print coords to trajectory every 500 steps (0.2 ps)
    }
}

sander_mdrun(input_top_path="input/structure.ions.parmtop",
            input_crd_path="output/sander.npt.rst",
            output_traj_path="output/sander.free.netcdf",
            output_rst_path="output/sander.free.rst",
            output_log_path="output/sander.free.log",
            properties=prop)

2022-07-08 12:31:20,747 [MainThread  ] [INFO ]  Creating c0540917-0aa1-48e3-9d7b-301611385ad9 temporary folder
2022-07-08 12:31:20,748 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2022-07-08 12:35:38,713 [MainThread  ] [INFO ]  sander -O -i c0540917-0aa1-48e3-9d7b-301611385ad9/sander.mdin -p input/structure.ions.parmtop -c output/sander.npt.rst -r output/sander.free.rst -o output/sander.free.log -x output/sander.free.netcdf

2022-07-08 12:35:38,715 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:35:38,718 [MainThread  ] [INFO ]  Removed: mdinfo, c0540917-0aa1-48e3-9d7b-301611385ad9


0

In [13]:
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

prop = {
    'mask': 'backbone',
    'reference': 'first'
}

cpptraj_rms(input_top_path="input/structure.ions.parmtop",
            input_traj_path="output/sander.free.netcdf",
            output_cpptraj_path=f"output/{pdbCode}_rms_first.dat",
            properties=prop)

2022-07-08 12:35:42,804 [MainThread  ] [INFO ]  Not using any container
2022-07-08 12:35:43,672 [MainThread  ] [INFO ]  cpptraj -i 31cffe2c-4fc5-445b-8dcb-9de9d867c7b9/instructions.in

2022-07-08 12:35:43,674 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:35:43,674 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 07/08/22 12:35:43
| Available memory: 1.643 GB

INPUT: Reading input from '31cffe2c-4fc5-445b-8dcb-9de9d867c7b9/instructions.in'
  [parm input/structure.ions.parmtop]
	Reading 'input/structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin output/sander.free.netcdf 1 -1 1]
	Reading 'output/sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, 

0

In [15]:
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

prop = {
    'mask': 'backbone',
    'reference': 'experimental'
}

cpptraj_rms(input_top_path="input/structure.ions.parmtop",
            input_traj_path="output/sander.free.netcdf",
            output_cpptraj_path=f"output/{pdbCode}_rms_exp.dat",
            input_exp_path="input/structure.leap.pdb",
            properties=prop)

2022-07-08 12:35:52,965 [MainThread  ] [INFO ]  Not using any container
2022-07-08 12:35:53,136 [MainThread  ] [INFO ]  cpptraj -i 3056d978-caa0-4477-98f3-06e1ac307399/instructions.in

2022-07-08 12:35:53,137 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 12:35:53,139 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 07/08/22 12:35:53
| Available memory: 1.669 GB

INPUT: Reading input from '3056d978-caa0-4477-98f3-06e1ac307399/instructions.in'
  [parm input/structure.ions.parmtop]
	Reading 'input/structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin output/sander.free.netcdf 1 -1 1]
	Reading 'output/sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, 

0

In [24]:
# Read RMS vs first snapshot data from file 
import numpy as np

with open(f"output/{pdbCode}_rms_first.dat",'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(f"output/{pdbCode}_rms_exp.dat",'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = np.arange(0.2,1.2,0.2),
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = np.arange(0.2,1.2,0.2),
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (Angstrom)")
                       )
}

plotly.offline.iplot(fig)

In [33]:
# Import module
from biobb_analysis.ambertools.cpptraj_image import cpptraj_image

prop = {
    'mask': 'solute',
    'format': 'trr'
}

# Create and launch bb
cpptraj_image(input_top_path='input/structure.ions.parmtop',
            input_traj_path="output/sander.free.netcdf",
            output_cpptraj_path=f"output/{pdbCode}_imaged_traj.trr",
            properties=prop)

2022-07-08 13:37:45,383 [MainThread  ] [INFO ]  Not using any container
2022-07-08 13:37:45,513 [MainThread  ] [INFO ]  cpptraj -i 955f917a-f916-472b-8bd5-3fe8dbb7da54/instructions.in

2022-07-08 13:37:45,516 [MainThread  ] [INFO ]  Exit code 0

2022-07-08 13:37:45,517 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 07/08/22 13:37:45
| Available memory: 150.067 MB

INPUT: Reading input from '955f917a-f916-472b-8bd5-3fe8dbb7da54/instructions.in'
  [parm input/structure.ions.parmtop]
	Reading 'input/structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin output/sander.free.netcdf 1 -1 1]
	Reading 'output/sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass

0

# Output for Docking